In [ ]:
import nbimporter
import pandas as pd
import json
import os
import re
%load_ext autoreload
%autoreload 2

In [ ]:
from step_2_ticker_converter import ticker_converter
from step_3_cloud_clean_data import cloud_clean_data
from step_1_historical_index import historical_data, get_quarter

In [ ]:
file_path = "historical_data.txt"
historical_index = historical_data(file_path)

In [ ]:
ticker_path = "ticker_converter.json"
ticker_conversion = ticker_converter(ticker_path)

In [ ]:
base_index = pd.merge(historical_index, ticker_conversion, on = "ticker", how = "left")
### JOIN CAUSED NAS - AS TICKER CONVERSION DIDNT HAVE COMPANY NAME AND CIK
### REMOVE THE NAS FROM THE JOIN
x = base_index.dropna()
## CREATE DF WITH THE NAS
y = base_index[base_index.isnull().any(axis=1)]
## DROP THE COLUMNS WE DONT NEED FROM NA DF
g = y[['quarter','ticker']]
## CSV HAS THE DATA WE NEED
missing_company = pd.read_csv('missing_companies.csv')
### MERGE BACK TO HAVE THE CIX AND COMPANY NAME 
z = pd.merge(g, missing_company, on = "ticker", how = "left")
### PAD THE CIK NUMBERS WITH ZEROS - 10 CHARCTERS
z['cik'] = z['cik'].astype(str).str.zfill(10)
## CONCAT BOTH DFS
df = pd.concat([x, z], axis=0, ignore_index=True)
# Split the 'quarter' column into two parts
df[['quarter', 'year']] = df['quarter'].str.split('-', expand=True)

# Convert 2-digit year to 4-digit
df['year'] = df['year'].apply(lambda x: '20' + x if int(x) < 50 else '19' + x)
df['year'] = df['year'].astype(int)

In [ ]:
#### testing on q1 
cloud_2022q1 = cloud_clean_data("2022q1")

In [ ]:

###  CLEANS DATA THIS CAN BE ADDED TO FUCNTIOM


# Convert to datetime if not already
cloud_2022q1['conformed_period_of_report'] = pd.to_datetime(cloud_2022q1['conformed_period_of_report'])

# Create the 'year' column
cloud_2022q1['year'] = cloud_2022q1['conformed_period_of_report'].dt.year

# Create the 'quarter' column in the format 'Q1', 'Q2', etc.
cloud_2022q1['quarter'] = 'Q' + cloud_2022q1['conformed_period_of_report'].dt.quarter.astype(str)
cloud_2022q1.loc[:, 'cik'] = cloud_2022q1['cik'].astype(str).str.zfill(10)

In [ ]:
##N CORE TABLE 

table_1 =pd.merge(df, cloud_2022q1, on =['cik','quarter','year'], how = "left")

In [ ]:
#### SEND TABLE UP TO BQ

from google.cloud import bigquery

def upload_core_csv_to_bq_(df):
    """
    Uploads parsed MDA data from EDGAR filings to BigQuery.
    Expects columns: cik, filename, management_discussion
    """
    # Rename columns to lowercase
    df.columns = df.columns.str.lower()


    try:
        # Keep only the required columns

        BQ_PROJECT_ID = 'sentiment-lewagon'
        BQ_DATASET_ID = 'sentiment_db'
        BQ_TABLE_ID = 'core'
        table_ref = f"{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{BQ_TABLE_ID}"

        client = bigquery.Client()

        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND"
        )

        job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
        job.result()

        print(f"✅ Uploaded {job.output_rows} rows to {table_ref}")

    except Exception as e:
        print(f"❌ Failed to upload DataFrame to BigQuery: {e}")


In [ ]:
### UPLOAD TEST

upload_core_csv_to_bq_(table_1)